In [ ]:
##Read the data collected then it will convert into a data frame and finally to a csv for analysis.

import pandas as pd

from io import StringIO

df = pd.read_csv("pHdata.txt",
                      delimiter = '/')
pd.read_csv(StringIO(df))
